In [1]:
from gernerate_data import split_train_and_test, load_SEG_data, load_SEG_data_for_test
from sklearn.metrics import classification_report, auc, roc_curve
# from models.Unet import UNet, UNet_IBA
# from models.att_Unet import att_UNet
# from models.r2_unet import r2_UNet
from keras.losses import categorical_crossentropy, sparse_categorical_crossentropy, binary_crossentropy
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, CSVLogger
from keras.optimizers import sgd, Adam
from utils.losses import dice_coef_loss, dice_coef, AUC, focal_tversky
from sklearn.model_selection import train_test_split
import tensorflow as tf
import cv2 as cv
import numpy as np
import os


Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [2]:
from keras.applications import VGG16
from keras.layers import *
from keras import Model, Sequential
def UNet(input_width, input_height, nClasses):
    assert input_height % 32 == 0
    assert input_width % 32 == 0

#     model = Sequential()
    inputShape = (height, width, 3)

    model = VGG16(include_top=False, weights='imagenet', input_shape=inputShape)
    for layer in model.layers:
        layer.trainable = True  # 涓嶈皟鏁翠箣鍓嶇殑鍗风Н灞傜殑鍙傛暟

    o = UpSampling2D((2, 2))(model.get_layer(name="block5_conv3").output)
    o = concatenate([model.get_layer(name="block4_conv3").output, o], axis=-1)
    o = Conv2D(512, (3, 3), padding="same")(o)
    o = BatchNormalization()(o)

    o = UpSampling2D((2, 2))(o)
    o = concatenate([model.get_layer(name="block3_conv3").output, o], axis=-1)
    o = Conv2D(256, (3, 3), padding="same")(o)
    o = BatchNormalization()(o)

    o = UpSampling2D((2, 2))(o)
    o = concatenate([model.get_layer(name="block2_conv2").output, o], axis=-1)
    o = Conv2D(128, (3, 3), padding="same")(o)
    # o = IBALayer()(o)
    o = BatchNormalization()(o)

    o = UpSampling2D((2, 2))(o)
    o = concatenate([model.get_layer(name="block1_conv2").output, o], axis=-1)
    o = Conv2D(64, (3, 3), padding="same")(o)
    o = BatchNormalization()(o)

    o = Conv2D(64, (3, 3), padding="same")(o)
    o = BatchNormalization()(o)

    o = Conv2D(nClasses, (3, 3), activation='softmax', padding='same', name='segmentation_output')(o)

    model = Model(inputs=model.input, outputs=o)
    return model



In [3]:
INIT_LR = 1e-4
EPOCHS = 200
BS = 8
width = 224
height = 224
depth = 3
target = (width, height)
shuffle = True
num_classes = 2
Name = 'UNet'
# Name = 'UNet_IBA_DESm'
preds_savePath = "./seg_prediction/LE/UNet"

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import keras
keras.backend.clear_session()


model = UNet(width, height, num_classes)
# model = att_UNet(width, height, num_classes)
# model = UNet_IBA(width, height, num_classes)
# model = r2_UNet(width, height, num_classes)
# weight_path = "UNet_IBA_DESm-031-0.9417.h5"
# model.load_weights(weight_path)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


2022-04-12 13:28:47.417333: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-04-12 13:28:47.449320: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200095000 Hz
2022-04-12 13:28:47.452817: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x680a060 executing computations on platform Host. Devices:
2022-04-12 13:28:47.452895: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2022-04-12 13:28:47.990709: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x68b6080 executing computations on platform CUDA. Devices:
2022-04-12 13:28:47.990767: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2022-04-12 13:28:47.991830: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found device 0 with properties: 
name: Tes

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [ ]:
print("------------------------------ Data loading ------------------------------------")

# trainX_dir = 'dataset/Dataset_BUSI_AN/train/images/'
# trainX_dir = 'dataset/DES_aug/train/images/'
trainX_dir = 'dataset/DESm/train/images/'
trainY_dir = trainX_dir.replace('images', 'masks')

valX_dir = trainX_dir.replace('train', 'val')
valY_dir = valX_dir.replace('images', 'masks')

testX_dir = trainX_dir.replace('train', 'test')
testY_dir = testX_dir.replace('images', 'masks')

train_X, train_Y, _ = load_SEG_data(trainX_dir, target=target, shuffle=True)
#(train_X, val_X, train_Y, val_Y) = train_test_split(train_X, train_Y, test_size=0.2)
val_X, val_Y, _ = load_SEG_data(valX_dir, target=target, shuffle=True)
test_X, test_Y, test_img_Names, test_orignal_images = load_SEG_data_for_test(testX_dir, target=target, shuffle=False)

print("train_X shape:", train_X.shape)
print("train_X shape:", train_Y.shape)
print("val_X shape:", val_X.shape)
print("val_Y shape:", val_Y.shape)
print("test_X shape:", test_X.shape)
print("test_Y shape:", test_Y.shape)

In [5]:
def binary_crossentropy(y_true, y_pred):
    e=1.0
    return K.mean(-(y_true*K.log(y_pred+K.epsilon())+
                    e*(1-y_true)*K.log(1-y_pred+K.epsilon())),
                  axis=-1)

def tversky(y_true, y_pred, smooth=1):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    alpha = 0.5
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)



# define callbacks
csv_logger = CSVLogger(Name+'.log')

reduce_lr = ReduceLROnPlateau(monitor='dice_coef', factor=0.1, patience=30, min_lr=1e-8, mode='auto', verbose=1)

checkpoint_period1 = ModelCheckpoint(Name + '-{epoch:03d}-{val_acc:.4f}.h5',
                                     monitor='val_acc', mode='auto', save_best_only='True')

checkpoint_period2 = ModelCheckpoint(Name + '-{epoch:03d}-{val_acc:.4f}.h5',
                                     monitor='val_acc', mode='auto', period=20)

# model.compile(loss=[tversky_dice_loss], optimizer=sgd, metrics=['accuracy', dice_coef])
# model.compile(optimizer=Adam(lr=INIT_LR), loss=binary_crossentropy, metrics=['accuracy', dice_coef, AUC])
model.compile(optimizer=Adam(lr=INIT_LR), loss=tversky_loss, metrics=['accuracy', dice_coef])


In [6]:
print("================================================ begin training ================================================")

history = model.fit(train_X, train_Y,
                    # initial_epoch=60,
                    batch_size=BS,
                    epochs=EPOCHS,
                    validation_data=(val_X, val_Y),
                    verbose=1,
                    callbacks=[checkpoint_period1, checkpoint_period2, reduce_lr, csv_logger],
                    shuffle=True)
                    

================================================ begin training ================================================
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 678 samples, validate on 101 samples
Epoch 1/200


InvalidArgumentError: Incompatible shapes: [401408] vs. [802816]
	 [[{{node loss/segmentation_output_loss/mul}}]]
	 [[{{node loss/mul}}]]

In [ ]:
print("================================================ Saving model ================================================")

# save the model
# model.save( Name + '.h5')
# model.load_weights('U-net_BUSI_224x224.h5', by_name = True)

In [ ]:
# evaluate the model
loss, acc, dice_coef = model.evaluate(test_X, test_Y, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', acc)
print('Test dice_coef:', dice_coef)

# test the model and gernerate results 
preds = model.predict(test_X, batch_size=4)


preds_s = model.predict(test_X, batch_size=8, verbose=1)
test_mask = test_Y.flatten()
pred_mask = preds_s.flatten()
fpr, tpr, thresholds = roc_curve(test_mask, pred_mask, pos_label=1)
roc_auc = auc(fpr, tpr)
print("AUC:", roc_auc)

In [ ]:
import matplotlib.pyplot as plt


N = np.arange(0, 2)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, history.history["loss"], label="train_loss")
plt.plot(N, history.history["val_loss"], label="val_loss")
plt.plot(N, history.history["acc"], label="train_acc")
plt.plot(N, history.history["val_acc"], label="val_acc")
# plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('./output/' + Name + '_plot.png')

In [ ]:
# def mask2gray(mask, input_type=None):
#     if input_type is 'pred':
#         mask0 = mask[:, :, 0]
#         mask1 = mask[:, :, 1]
#         mask = mask1 - mask0
#         print("mask.shape: ", mask.shape)
#         #mask = np.argmax(mask, axis=-1)
#         # print( mask[mask > 0.5] )
#         mask[mask > 0] = 255
#         mask[mask <= 0] = 0
#     mask = mask.astype(dtype=np.uint8)
#     rst = mask.copy()
#     cv.normalize(mask, rst, alpha=0, beta=255, norm_type=cv.NORM_MINMAX)
#     return rst

# print("------------------------------ Saving predictions ------------------------------------")

# img_size = (width, height)
# for i in range(test_X.shape[0]):
#     img = test_orignal_images[i]

#     gt = test_Y[i, :, :, 1]
#     gt = mask2gray(gt)

#     pred = preds[i]
#     prediction = mask2gray(pred, input_type='pred')

#     save_img = np.zeros([img_size[0], img_size[1]*3, 3], dtype=np.uint8)
#     save_img[:, 0:img_size[0], :] = img[:, :, ::-1]
#     save_img[:, img_size[0]:img_size[1] * 2, :] = cv.cvtColor(gt, cv.COLOR_GRAY2RGB)
#     save_img[:, img_size[0] * 2:img_size[1] * 3, :] = cv.cvtColor(prediction, cv.COLOR_GRAY2RGB)

#     savePath = os.path.join(preds_savePath, "joint")

#     if not os.path.exists(savePath):
#         os.makedirs(savePath)

#     cv.imwrite(savePath + "{0}".format(test_img_Names[i]), save_img)

# for i in range(test_X.shape[0]):

#     img = test_orignal_images[i]

#     gt = test_Y[i, :, :, 1]
#     gt = mask2gray(gt)
#     gt = cv.cvtColor(gt, cv.COLOR_GRAY2RGB)

#     pred_mask = preds[i]
#     pred_mask = mask2gray(pred_mask, input_type='pred_mask')

#     savePath = os.path.join(preds_savePath, "Imgs/")
#     saveMaskPath = os.path.join(preds_savePath, "Mask/")
#     savePredPath = os.path.join(preds_savePath, "Pred/")

#     if not os.path.exists(savePath):
#         os.makedirs(savePath)
#     if not os.path.exists(saveMaskPath):
#         os.makedirs(saveMaskPath)
#     if not os.path.exists(savePredPath):
#         os.makedirs(savePredPath)

#     cv.imwrite(os.path.join(savePath, "{0}".format(test_img_Names[i])), img)
#     cv.imwrite(os.path.join(saveMaskPath, "{0}".format(test_img_Names[i])), gt)
#     cv.imwrite(os.path.join(savePredPath, "{0}".format(test_img_Names[i])), pred)

